In [12]:
import bs4 as bs
import requests
import pickle
import os
import datetime as dt
from pandas_datareader import data as pdr
import pandas as pd

In [7]:
with open('sp500tickers.pickle','rb') as f:
    tickers = pickle.load(f)

In [12]:
def get_finance_data(ticker):
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    resp = requests.get('http://basic.10jqka.com.cn/{}/finance.html?stockpage=1'.format(ticker),headers=headers)
    soup = bs.BeautifulSoup(resp.text)
    table = soup.find('p', {'id':'keyindex'})
    df = pd.DataFrame()
    a = table.text
    a1 = a.split('report')[1]
    a2 = a1.split('[')[3:]
    index = 0;
    for zhibiao in a2:
        index = index + 1
        a3 = zhibiao.split('"')
        n = len(a3)
        df[index] = ''
        for i in range(n//2):
            df.loc[i,index]=(a3[2*i+1])
    return df

In [22]:
if not os.path.exists('finance'):
    os.makedirs('finance')
for ticker in tickers:
    if os.path.exists('finance/{}.csv'.format(ticker)):
        df = get_finance_data(ticker.replace('-','.'))
        for i in range(len(df)):
            df.loc[i,'year'] = 2020-(i+2)//4
            df.loc[i,'quart'] = 4-(i+2)%4
        df.to_csv('finance/{}.csv'.format(ticker),index=False)
    else:
        print('Already have {}'.format(ticker))